In [11]:
import pandas as pd
from sklearn import *
import numpy as np

In [12]:
df = pd.read_csv("/data/credit-default.csv")

In [13]:
df.head()

,checking_balance,months_loan_duration,credit_history,purpose,amount,savings_balance,employment_length,installment_rate,personal_status,other_debtors,...,property,age,installment_plan,housing,existing_credits,default,dependents,telephone,foreign_worker,job
0,< 0 DM,6,critical,radio/tv,1169,unknown,> 7 yrs,4,single male,none,...,real estate,67,none,own,2,1,1,yes,yes,skilled employee
1,1 - 200 DM,48,repaid,radio/tv,5951,< 100 DM,1 - 4 yrs,2,female,none,...,real estate,22,none,own,1,2,1,none,yes,skilled employee
2,unknown,12,critical,education,2096,< 100 DM,4 - 7 yrs,2,single male,none,...,real estate,49,none,own,1,1,2,none,yes,unskilled resident
3,< 0 DM,42,repaid,furniture,7882,< 100 DM,4 - 7 yrs,2,single male,guarantor,...,building society savings,45,none,for free,1,1,2,none,yes,skilled employee
4,< 0 DM,24,delayed,car (new),4870,< 100 DM,1 - 4 yrs,3,single male,none,...,unknown/none,53,none,for free,2,2,2,none,yes,skilled employee


In [14]:
df.default.value_counts()

1    700
2    300
Name: default, dtype: int64

In [15]:
# dependent variable/target varible/label
label = "default"

# independepent variables/features/predictors
X = df.drop(columns=[label])
indices = np.arange(len(df))


# vector for target variable
y = df[label]

# one hot encoding to conver categorical features into numeric
# drop_first: remove first value of a categorical feature ... it is redundant
X = pd.get_dummies(X, drop_first=True)

# Creating trainnig and test sets
# test_size is 30% of the whole
# random_state: to reprduce the same combination of training and test records
X_train, X_test, y_train, y_test, indices_train, indices_test = model_selection.train_test_split(X.values,                       
                                    y, indices, test_size = 0.3, random_state = 1)


pipe = pipeline.Pipeline([
    ("poly", preprocessing.PolynomialFeatures(degree=1, include_bias=False)),
    ("scaler", preprocessing.StandardScaler()),
    ("est", linear_model.LogisticRegression(solver="liblinear"))
])

pipe.fit(X_train, y_train)

# prediction on training and test data
y_train_pred = pipe.predict(X_train)
y_test_pred = pipe.predict(X_test)

print(metrics.confusion_matrix(y_test,y_test_pred))
print("Training accuracy", metrics.accuracy_score(y_train, y_train_pred))
print("Test accuracy", metrics.accuracy_score(y_test, y_test_pred))
print("Training precision", metrics.precision_score(y_train, y_train_pred))
print("Test precision", metrics.precision_score(y_test, y_test_pred))
print("Training recall", metrics.recall_score(y_train, y_train_pred))
print("Test recall", metrics.recall_score(y_test, y_test_pred))

[[188  26]
 [ 47  39]]
Training accuracy 0.7957142857142857
Test accuracy 0.7566666666666667
Training precision 0.8304431599229287
Test precision 0.8
Training recall 0.8868312757201646
Test recall 0.8785046728971962


In [16]:
df["prediction"] = 0

In [17]:
df.loc[indices_train, "prediction"] = y_train_pred
df.loc[indices_test, "prediction"] = y_test_pred

In [18]:
df.prediction.value_counts()

1    754
2    246
Name: prediction, dtype: int64

In [30]:
df = pd.read_csv("/data/credit-default.csv")

In [31]:
# dependent variable/target varible/label
label = "default"

# independepent variables/features/predictors
X = df.drop(columns=[label])

# vector for target variable
y = df[label]

# one hot encoding to conver categorical features into numeric
# drop_first: remove first value of a categorical feature ... it is redundant
X = pd.get_dummies(X, drop_first=True)

# Creating trainnig and test sets
# test_size is 30% of the whole
# random_state: to reprduce the same combination of training and test records
X_train, X_test, y_train, y_test = model_selection.train_test_split(X.values,                       
                                    y,  test_size = 0.3, random_state = 1)


pipe = pipeline.Pipeline([
    ("poly", preprocessing.PolynomialFeatures(degree=1, include_bias=False)),
    ("scaler", preprocessing.StandardScaler()),
    ("est", tree.DecisionTreeClassifier(max_depth=4, random_state=1))
])

pipe.fit(X_train, y_train)

# prediction on training and test data
y_train_pred = pipe.predict(X_train)
y_test_pred = pipe.predict(X_test)

print(metrics.confusion_matrix(y_test,y_test_pred))
print("Training accuracy", metrics.accuracy_score(y_train, y_train_pred))
print("Test accuracy", metrics.accuracy_score(y_test, y_test_pred))
print("Training precision", metrics.precision_score(y_train, y_train_pred))
print("Test precision", metrics.precision_score(y_test, y_test_pred))
print("Training recall", metrics.recall_score(y_train, y_train_pred))
print("Test recall", metrics.recall_score(y_test, y_test_pred))

[[186  28]
 [ 46  40]]
Training accuracy 0.7657142857142857
Test accuracy 0.7533333333333333
Training precision 0.8108108108108109
Test precision 0.8017241379310345
Training recall 0.8641975308641975
Test recall 0.8691588785046729


In [32]:
est = pipe.steps[-1][-1]
est

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=4,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=1,
            splitter='best')

In [33]:
from sklearn.tree import export_graphviz
export_graphviz(est, out_file = "tree.dot", feature_names = X.columns, filled=True)
!dot -Tpng tree.dot -o tree.png


In [34]:
importance = pd.DataFrame({"feature": X.columns, "importance":  est.feature_importances_})
importance.sort_values("importance", ascending=False)

,feature,importance
9,checking_balance_unknown,0.401680
0,months_loan_duration,0.231672
11,credit_history_fully repaid,0.079479
24,savings_balance_< 100 DM,0.065018
4,age,0.047438
12,credit_history_fully repaid this bank,0.046693
39,installment_plan_none,0.045654
13,credit_history_repaid,0.041213
35,other_debtors_none,0.020657
1,amount,0.020495
